In [1]:
from transformer_lens import HookedTransformer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_name = "gpt2"
model = HookedTransformer.from_pretrained(model_name, device = device)

/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ubuntu/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


cuda


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2 into HookedTransformer


In [2]:
from sparsifiers import SAE
from huggingface_hub import hf_hub_download

layer = 6
# e2e_name = "https://huggingface.co/apollo-research/e2e-saes-gpt2/blob/main/downstream_similar_ce_layer_6.pt"
downstream_name = f"downstream_similar_ce_layer_{layer}.pt"
# downstream_name = f"downstream_similar_l0_layer_{layer}.pt"
local_name = f"local_similar_ce_layer_{layer}.pt"
# local_name = f"local_similar_l0_layer_{layer}.pt"

activation_name = f"transformer.h.{layer}"
model_id = "apollo-research/e2e-saes-gpt2"

In [3]:
def load_sae(model_id, hh_file_location):
    ae_download_location = hf_hub_download(repo_id=model_id, filename=hh_file_location)

    sae_list = torch.load(ae_download_location)

    # Rename the keys in the state dictionary
    def rename_keys(state_dict):
        renamed_state_dict = {}
        for key, value in state_dict.items():
            if key == 'encoder.weight':
                renamed_state_dict['encoder.0.weight'] = value
            elif key == 'encoder.bias':
                renamed_state_dict['encoder.0.bias'] = value
            else:
                renamed_state_dict[key] = value
        return renamed_state_dict

    # Create an instance of the SAE model with the extracted parameters
    n_dict_components, input_size = sae_list['encoder.weight'].shape
    sae = SAE(input_size=input_size, n_dict_components=n_dict_components)

    # Rename the keys in the state dictionary
    renamed_sae_list = rename_keys(sae_list)

    # Load the state dictionary into the model instance
    sae.load_state_dict(renamed_sae_list)
    return sae

sae_local = load_sae(model_id, local_name).to(device)
sae_downstream = load_sae(model_id, downstream_name).to(device)

In [4]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from einops import rearrange

def download_dataset(dataset_name, tokenizer, max_length=256, num_datapoints=None):
    if(num_datapoints):
        split_text = f"train[:{num_datapoints}]"
    else:
        split_text = "train"
    dataset = load_dataset(dataset_name, split=split_text).map(
        lambda x: tokenizer(x['text']),
        batched=True,
    ).filter(
        lambda x: len(x['input_ids']) > max_length
    ).map(
        lambda x: {'input_ids': x['input_ids'][:max_length]}
    )
    return dataset

dataset_name = "stas/openwebtext-10k"
max_seq_length = 40
print(f"Downloading {dataset_name}")
dataset = download_dataset(dataset_name, tokenizer=model.tokenizer, max_length=max_seq_length, num_datapoints=None) # num_datapoints grabs all of them if None

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# now run through the model & grab activations to get dead features
def get_dictionary_activations(model, dataset, cache_name, max_seq_length, autoencoder, batch_size=32):
    device = model.cfg.device
    num_features, d_model = autoencoder.encoder[0].weight.shape
    datapoints = dataset.num_rows
    dictionary_activations = torch.zeros((datapoints*max_seq_length, num_features))
    token_list = torch.zeros((datapoints*max_seq_length), dtype=torch.int64)
    with torch.no_grad(), dataset.formatted_as("pt"):
        dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
        for i, batch in enumerate(tqdm(dl)):
            batch = batch.to(device)
            token_list[i*batch_size*max_seq_length:(i+1)*batch_size*max_seq_length] = rearrange(batch, "b s -> (b s)")
            # with Trace(model, cache_name) as ret:
            #     _ = model(batch).logits
            #     internal_activations = ret.output
            #     # check if instance tuple
            #     if(isinstance(internal_activations, tuple)):
            #         internal_activations = internal_activations[0]
            _, cache = model.run_with_cache(batch)
            # print(cache)
            internal_activations = cache[cache_name]
            batched_neuron_activations = rearrange(internal_activations, "b s n -> (b s) n" )
            batched_dictionary_activations = autoencoder.encoder(batched_neuron_activations)
            dictionary_activations[i*batch_size*max_seq_length:(i+1)*batch_size*max_seq_length,:] = batched_dictionary_activations.cpu()
    return dictionary_activations, token_list

batch_size = 128
activation_name = f"blocks.{layer}.hook_resid_post"

In [6]:
local_dictionary_activations, local_token_list = get_dictionary_activations(model, dataset, activation_name, max_seq_length, sae_local, batch_size=batch_size)
downstream_dictionary_activations, downstream_token_list = get_dictionary_activations(model, dataset, activation_name, max_seq_length, sae_downstream, batch_size=batch_size)

100%|██████████| 79/79 [00:45<00:00,  1.72it/s]


In [7]:
do_decoder = True

if(do_decoder):
    dec_local = sae_local.decoder.weight.data
    dec_downstream = sae_downstream.decoder.weight.data
else:
    dec_local = sae_local.encoder[0].weight.data.T
    dec_downstream = sae_downstream.encoder[0].weight.data.T

dec_local = dec_local / dec_local.norm(dim=0, keepdim=True)
dec_downstream = dec_downstream / dec_downstream.norm(dim=0, keepdim=True)

In [8]:
# Compute cos sim
cos_sim = torch.mm(dec_local.T, dec_downstream)

# Find pairs of local & downstream features w/ cos sim > 0.9
threshold = 0.9
high_cos_sim_pairs = torch.nonzero(cos_sim > threshold)

In [9]:
# Sanity check (cos_sim has shape feature x feature)
cos_sim.shape

torch.Size([46080, 46080])

In [10]:
# Sanity check (use high_cos_sim_pairs to index into cos_sim gives values > threshold)
cos_sim[high_cos_sim_pairs[:, 0], high_cos_sim_pairs[:, 1]]

tensor([0.9144, 0.9669, 0.9729,  ..., 0.9210, 0.9125, 0.9211], device='cuda:0')

In [11]:
len(high_cos_sim_pairs)

7750

In [12]:
def get_activation_overlap(local_active, downstream_active):
    # Get active inputs for both local & downstream
    both_active = local_active & downstream_active
    
    # Calculate counts
    local_active_count = local_active.sum().item()
    downstream_active_count = downstream_active.sum().item()
    both_active_count = both_active.sum().item()
    
    # Calculate percentages
    if local_active_count > 0:
        percent_of_local = (both_active_count / local_active_count) * 100
    else:
        percent_of_local = 0
    
    if downstream_active_count > 0:
        percent_of_downstream = (both_active_count / downstream_active_count) * 100
    else:
        percent_of_downstream = 0
    
    return {
        "percent_of_local": percent_of_local,
        "percent_of_downstream": percent_of_downstream,
        "local_active_count": local_active_count,
        "downstream_active_count": downstream_active_count,
        "both_active_count": both_active_count
    }

In [13]:
def find_activation_contexts(activations, token_list, top_k=20):
    top_indices = torch.argsort(activations, descending=True)[:top_k]
    contexts = [token_list[idx-2:idx+3].tolist() for idx in top_indices]
    return contexts

In [14]:
subset_examples = []

for local_idx, downstream_idx in tqdm(high_cos_sim_pairs):
    local_activations = local_dictionary_activations[:, local_idx]
    downstream_activations = downstream_dictionary_activations[:, downstream_idx]
    
    # Check if local feature activates on a subset of downstream feature
    # Count as active if above mean + std
    local_active = local_activations > local_activations.mean() + local_activations.std()
    downstream_active = downstream_activations > downstream_activations.mean() + downstream_activations.std()
    
    if local_active.sum() < downstream_active.sum() and (local_active & downstream_active).sum() / local_active.sum() > 0.9:
        print(get_activation_overlap(local_active, downstream_active))
        local_contexts = find_activation_contexts(local_activations, local_token_list)
        downstream_contexts = find_activation_contexts(downstream_activations, downstream_token_list)
        
        subset_examples.append({
            'local_idx': local_idx.item(),
            'downstream_idx': downstream_idx.item(),
            'local_contexts': local_contexts,
            'downstream_contexts': downstream_contexts
        })

  0%|          | 24/7750 [00:00<01:43, 74.85it/s]

{'percent_of_local': 93.33333333333333, 'percent_of_downstream': 58.333333333333336, 'local_active_count': 105, 'downstream_active_count': 168, 'both_active_count': 98}
{'percent_of_local': 93.04347826086956, 'percent_of_downstream': 85.6, 'local_active_count': 115, 'downstream_active_count': 125, 'both_active_count': 107}
{'percent_of_local': 94.79166666666666, 'percent_of_downstream': 77.11864406779661, 'local_active_count': 96, 'downstream_active_count': 118, 'both_active_count': 91}


  1%|          | 54/7750 [00:00<01:06, 116.58it/s]

{'percent_of_local': 92.39766081871345, 'percent_of_downstream': 49.52978056426332, 'local_active_count': 171, 'downstream_active_count': 319, 'both_active_count': 158}


  1%|          | 78/7750 [00:00<01:07, 114.14it/s]

...

{'percent_of_local': 91.47727272727273, 'percent_of_downstream': 53.666666666666664, 'local_active_count': 176, 'downstream_active_count': 300, 'both_active_count': 161}


 99%|█████████▉| 7670/7750 [00:54<00:01, 58.62it/s]

{'percent_of_local': 90.38461538461539, 'percent_of_downstream': 33.098591549295776, 'local_active_count': 52, 'downstream_active_count': 142, 'both_active_count': 47}


100%|██████████| 7750/7750 [00:55<00:00, 139.22it/s]

{'percent_of_local': 99.27007299270073, 'percent_of_downstream': 63.55140186915887, 'local_active_count': 137, 'downstream_active_count': 214, 'both_active_count': 136}
{'percent_of_local': 91.73553719008265, 'percent_of_downstream': 47.43589743589743, 'local_active_count': 121, 'downstream_active_count': 234, 'both_active_count': 111}


In [18]:
print(f"{len(subset_examples)} local feature that is a subset of downstream feature")

189 local feature that is a subset of downstream feature


In [19]:
# Print out the first 5 examples
for i, example in enumerate(subset_examples[:5]):
    print(f"\nExample {i+1}:")

    print(f"Local feature {example['local_idx']} contexts:")
    for context in example['local_contexts']:
        print(" ".join(model.to_string(context)))
    
    print(f"Downstream feature {example['downstream_idx']} contexts:")
    for context in example['downstream_contexts']:
        print(" ".join(model.to_string(context)))


Example 1:
Local feature 70 contexts:
� s   p r o - t r a n s
  w e e k e n d   a t   p r o - T r u m p
  o f f i c i a l   a n d   p r o - T r u m p
  t i r e l e s s l y   f o r   p r o - l
  b l a m i n g   a   p r o - T r u m p
  p r e s s e s   h i s   p r o - i m m i g r a t i o n
  c o n t r o l   o f   p r o - R u s s i a n
  m a k i n g   t h e   p r o j e c
  s a y i n g   t h a t   p r o - g o v e r n m e n t
  a f t e r   a   p r o - T r u m p
  T u e s d a y   w i t h   p r o - R u s s i a n
H o w   d o   p r o   f a n s   s t a c k
A f t e r   t h e   p r o - t r a n s
  m y t h ,   p r o p p e d   u p
  a   r a d i c a l l y   p r o - g r o w t h
  a l w a y s   b e   p r o - w o m a n
  o f   c o n s e r v a t i v e   p r o - f a m i l y
  a d v o c a t e   s a y s   p r o - a b o r t i o n
  t h e y   c a n   p r o c r e a t e   a n d
  s h e   i s   p r o   w o m a n   w h e n
Downstream feature 4456 contexts:
  w e e k e n d   a t   p r o - T r u m p
  T u e s d a y